In [0]:
%sql
USE retailpulse_lite;

-- SILVER
CREATE OR REPLACE TABLE silver_customers AS
SELECT
  customer_id,
  lower(email) AS email,
  country,
  to_timestamp(created_at) AS created_at,
  status
FROM bronze_customers
WHERE customer_id IS NOT NULL;

CREATE OR REPLACE TABLE silver_products AS
SELECT
  product_id,
  category,
  cast(price AS DECIMAL(10,2)) AS price,
  cast(active AS BOOLEAN) AS active
FROM bronze_products
WHERE product_id IS NOT NULL;

CREATE OR REPLACE TABLE silver_orders AS
SELECT
  order_id,
  customer_id,
  to_timestamp(order_ts) AS order_ts,
  status,
  channel
FROM bronze_orders
WHERE order_id IS NOT NULL;

CREATE OR REPLACE TABLE silver_order_items AS
SELECT
  order_id,
  cast(line_id AS INT) AS line_id,
  product_id,
  cast(quantity AS INT) AS quantity
FROM bronze_order_items
WHERE order_id IS NOT NULL AND product_id IS NOT NULL;

-- CDC-lite: upsert customers, insert new orders
MERGE INTO silver_customers tgt
USING (
  SELECT customer_id, lower(email) AS email, country, to_timestamp(created_at) AS created_at, status
  FROM bronze_customers_incr
) src
ON tgt.customer_id = src.customer_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;

MERGE INTO silver_orders tgt
USING (
  SELECT order_id, customer_id, to_timestamp(order_ts) AS order_ts, status, channel
  FROM bronze_orders_incr
) src
ON tgt.order_id = src.order_id
WHEN NOT MATCHED THEN INSERT *;

-- GOLD
CREATE OR REPLACE TABLE fact_order_item AS
SELECT
  o.order_id, o.customer_id, o.order_ts, o.channel, o.status,
  i.line_id, i.product_id, i.quantity,
  p.category, p.price,
  (i.quantity * p.price) AS line_amount
FROM silver_orders o
JOIN silver_order_items i ON o.order_id = i.order_id
JOIN silver_products p ON i.product_id = p.product_id
WHERE p.active = true;

CREATE OR REPLACE TABLE mart_daily_revenue AS
SELECT
  date(order_ts) AS order_date,
  channel,
  sum(line_amount) AS revenue,
  count(distinct order_id) AS orders,
  count(distinct customer_id) AS customers
FROM fact_order_item
WHERE status IN ('placed','shipped','delivered')
GROUP BY 1,2;

-- DQ checks
SELECT customer_id, count(*) c
FROM silver_customers
GROUP BY customer_id
HAVING c > 1;

SELECT count(*) AS orphan_items
FROM silver_order_items i
LEFT JOIN silver_orders o ON i.order_id = o.order_id
WHERE o.order_id IS NULL;

SELECT * FROM mart_daily_revenue ORDER BY order_date, channel;


In [0]:
%sql
--SELECT count(*) FROM silver_orders;
SELECT status, count(*) FROM silver_customers GROUP BY status;